# Day 10 (2023-12-10), Exercise a

In [1]:
with open("input_10.txt", "r") as fh:
    lines = fh.readlines()
# Remove \n characters
lines = [l.strip() for l in lines]
assert len(lines)==len(lines[0]), "Assuming square input"

In [2]:
# Symbols:
# | => NS, - => EW, 7 => SW, J => NW, L => NE, F => SE
def next_coord(char, cur_coord, prev_coord):
    x1, y1 = cur_coord
    x0, y0 = prev_coord
    match char:
        case '|':
            next_coord = (x1, y1+1 if y1>y0 else y1-1)
        case '-':
            next_coord = (x1+1 if x1>x0 else x1-1, y1)
        case '7':
            next_coord = (x1 if x1!=x0 else x1-1, y1+1 if y1==y0 else y1)
        case 'J':
            next_coord = (x1 if x1!=x0 else x1-1, y1-1 if y1==y0 else y1)
        case 'L':
            next_coord = (x1 if x1!=x0 else x1+1, y1-1 if y1==y0 else y1)
        case 'F':
            next_coord = (x1 if x1!=x0 else x1+1, y1+1 if y1==y0 else y1)
        case _:
            assert False, "Should never hit the default case, since we are in a loop."
    return next_coord

In [3]:
# find S (x[right positive] and y[down positive] coordinates)
for y, l in enumerate(lines):
    x = l.find('S')
    if x > 0:
        break

# find first step
x_prev, y_prev = x, y
if lines[y][x+1] in ['-', '7', 'J']:
    x += 1
    current_char = lines[y][x+1]
elif lines[y][x-1] in ['-', 'L', 'F']:
    x -= 1
    current_char = lines[y][x-1]
elif lines[y+1][x] in ['|', 'J', 'L']:
    y += 1
    current_char = lines[y+1][x]
elif lines[y-1][x] in ['|', '7', 'F']:
    y -= 1
    current_char = lines[y-1][x]

# build visualization loop (for exercise b) only containing loop
hist = {(x_prev, y_prev): 'S', (x, y): current_char}

# go through entire loop
steps_return_trip = 1
while current_char != 'S':
    x_new, y_new = next_coord(current_char, (x, y), (x_prev, y_prev))
    steps_return_trip += 1
    x_prev, y_prev = x, y
    x, y = x_new, y_new
    current_char = lines[y][x]
    hist[(x,y)] = current_char

print(f"Furthest point is at: {steps_return_trip/2}")    

Furthest point is at: 6886.0


# Day 10 (2023-12-10), Exercise b

In [4]:
# Approach: walk the loop CCW, and check all tiles at left, until another edge is found.

# Get directions for checking enclosed tiles
# LEFT positive
dirs_inwards = {}
hist_items = list(hist.items())
k0, _ = hist_items[0]
for k, c in hist_items[1:]:
    dx = k[0]-k0[0]
    dy = k[1]-k0[1]
    match c:
        case '|':
            dirs_inwards[k] = (dy, 0)
        case '-':
            dirs_inwards[k] = (0, -dx)
        case '7':
            dirs_inwards[k] = (1, -1) if dx != 0 else (-1, 1)
        case 'F':
            dirs_inwards[k] = (1, 1) if dx != 0 else (-1, -1)
        case 'J':
            dirs_inwards[k] = (-1, -1) if dx != 0 else (1, 1)
        case 'L':
            dirs_inwards[k] = (-1, 1) if dx != 0 else (1, -1)
    k0 = k

In [5]:
# Check all empty tiles on inside of trajectory.
# Use set for enclosed squares, to exclude duplicates
enclosed_squares = set()
# Left-right as inside by trial and error, knowing that most empty tiles are not on inside
lr = -1 
# Loop through entire path. Dirs_inwards contains unit vectors pointing inward.
for k, d in dirs_inwards.items():
    xi, yi = k
    finished = False # To break out of both loops 
    while len(lines)>xi>=0:
        while len(lines)>yi>=0:
            if (xi, yi) != k: # Ignore current path itself
                if (xi, yi) in hist.keys():
                    finished = True
                    break
                enclosed_squares.add((xi,yi))
            yi += lr*d[1]
            if d[1] == 0:
                break
        xi += lr*d[0]
        if d[0] == 0 or finished:
            break
len(enclosed_squares)

371